# Forest Fire Model

Make it possible to set the probability parameters p, f, q. Start with an empty field (all cells
are in state Ashes).
Write the number of cells for each state (Ashes, Tree, Fire) for each time step into a file
(three values per line, ASCII format, separated by blanks).

q - rate of induced growth\
f - rate of spontaneous fire\
p - rate of spontaneous growth

In [2]:
import random

In [3]:
q = 1
f = 0.1
p = 0.5

In [4]:
rows, cols = (101, 82)
automaton = [[0]*cols]*rows

### Auxilary functions

In [6]:
def colored_string(numbers):
    color_map = {"0": "\033[0;33m", "1": "\033[32m", "2": "\033[31m"}
    colored_str = ""
    for n in numbers:
        if str(n) in color_map:
            colored_str += color_map[str(n)] + str(n) + "\033[0m"
        else:
            colored_str += str(n)
    return colored_str

In [7]:
print(colored_string("000111222"))

000111222


Let 0 be ashes, 1 - forest and 2 - fire

In [ ]:
for rows in automaton:
    for cols in rows:
        pass